# Title
[]()

In [8]:
import requests
import json
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
from silvhua import *
import pickle
import os

In [12]:
with open("credentials.json") as f:
    credentials = json.load(f)
# user_id = os.environ['fb_user_id']

In [4]:
def get_user_ig_post(ig_user_id, access_token, pages=5, filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    user_id = str(ig_user_id)
    url_root = "https://graph.facebook.com/v15.0/"
    # url_without_token = f'{url_root}{ig_user_id}/media?fields=caption%2Ccomments_count%2Cis_shared_to_feed%2Clike_count%2Cmedia_type%2Cmedia_url%2Cmedia_product_type%2Cpermalink%2Ctimestamp%2Ccomments&access_token='
    # url_without_token = f'{url_root}{ig_user_id}/media?fields=caption,comments_count,is_shared_to_feed,like_count,media_type,media_url,media_product_type,permalink,timestamp,comments&access_token='
    url_without_token = f'{url_root}{ig_user_id}/media?access_token='
    url = url_without_token+access_token
    print(url_without_token)
    response_json_dict = dict()
    df_list = []
    for page in range(1,pages+1):
        response = requests.get(url)
        print('Response status code: ',response.status_code)
        response_json_dict[page] = response.json()
        df_list.append(json_normalize(response_json_dict[page], record_path='data'))
        try:
            url = response_json_dict[page]['paging']['next']
        except: 
            pass
    df = pd.concat(df_list)
    print('Number of posts:',len(df))
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

df_test, response_json_test = get_user_ig_post(credentials['ig_user_id'], 
        credentials['access_token'], pages=2, filename='my_ig_posts_2p_2023-01-23')


https://graph.facebook.com/v15.0/17841403231458630/media?access_token=
Response status code:  200
Response status code:  200
Number of posts: 50
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/my_ig_posts_2p_2023-01-23.csv
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/my_ig_posts_2p_2023-01-23.sav


In [13]:
df_test_am, response_json_test_am = get_user_ig_post(credentials['am_ig_user_id'], 
        credentials['am2_access_token'], pages=2, filename='am_ig_posts_2p_2023-01-25')


https://graph.facebook.com/v15.0/17841401256046961/media?access_token=
Response status code:  200
Response status code:  200
Number of posts: 50
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/am_ig_posts_2p_2023-01-25.csv
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/am_ig_posts_2p_2023-01-25.sav


In [104]:
df_test_am.columns

Index(['caption', 'like_count', 'comments_count', 'media_type',
       'media_product_type', 'media_url', 'permalink', 'timestamp', 'id',
       'comments.data', 'thumbnail_url', 'comments.paging.cursors.after',
       'comments.paging.next'],
      dtype='object')

# `get_user_ig_post_text`

In [107]:
def get_user_ig_post_text(ig_user_id, access_token, pages=5, filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ SH 2023-01-25 21:52
    Pull the media from a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - pages: Number of pages of results to retrieve starting in reverse chronological order.
            Each page of results provides 25 posts.
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information (1 post/reel per row)
            - caption
            - like count
            - comments count
            - top-level comments: If the amount of comments is high (>53 commments), it will provide the endpoints, 
                it will return additional endpoints for additional comments ('comments.paging.*').
            - media type: video or image
            - media product type: reels or feed
            - media URL (not available for all reels)
            - permalink
            - timestamp
            - post id
            - comments.data (list of dictionaries): timestamp, username, text, like count, and comment id.
            - thumbnail URL of videos
        - response_json: JSON object with each page number of results as the key (starting with 1)

    """
    user_id = str(ig_user_id)
    url_root = "https://graph.facebook.com/v15.0/"
    # url_without_token = f'{url_root}{ig_user_id}/media?fields=caption%2Clike_count%2Ccomments_count%2Cmedia_type%2Cmedia_product_type%2Cmedia_url%2Cpermalink%2Ctimestamp%2Ccomments%7Btimestamp%2Cusername%2Ctext%2Clike_count%7D%2Cthumbnail_url&access_token='
    url_without_token = f'{url_root}{ig_user_id}/media?fields=timestamp%2Ccaption%2Clike_count%2Ccomments_count%2Cmedia_type%2Cmedia_product_type%2Cmedia_url%2Cpermalink%2Cid%2Cthumbnail_url%2Ccomments%7Btimestamp%2Ctext%2Cusername%2Clike_count%2Creplies%7Btimestamp%2Ctext%2Cusername%2Clike_count%7D%7D&access_token='
    url = url_without_token+access_token
    print(url_without_token)
    response_json_dict = dict()
    df_list = []
    for page in range(1,pages+1):
        response = requests.get(url)
        print('Response status code: ',response.status_code)
        response_json_dict[page] = response.json()
        df_list.append(json_normalize(response_json_dict[page], record_path='data'))
        try:
            url = response_json_dict[page]['paging']['next']
        except: 
            pass
    df = pd.concat(df_list)
    print('Number of posts:',len(df))
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

df_test_am, response_json_test_am = get_user_ig_post_text(credentials['am_ig_user_id'], 
        credentials['am2_access_token'], pages=2, filename='am_ig_posts_2022-01-25')


https://graph.facebook.com/v15.0/17841401256046961/media?fields=timestamp%2Ccaption%2Clike_count%2Ccomments_count%2Cmedia_type%2Cmedia_product_type%2Cmedia_url%2Cpermalink%2Cid%2Cthumbnail_url%2Ccomments%7Btimestamp%2Ctext%2Cusername%2Clike_count%2Creplies%7Btimestamp%2Ctext%2Cusername%2Clike_count%7D%7D&access_token=
Response status code:  200
Response status code:  200
Number of posts: 50
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/am_ig_posts_2022-01-25.csv
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/am_ig_posts_2022-01-25.sav


In [108]:
df_test_am.head()

,timestamp,caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,thumbnail_url,comments.paging.cursors.after,comments.paging.next
0,2023-01-25T20:37:00+0000,Why I love my muscles 🫶\n\nWhat you see:\n✅ A ...,62,1,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/Cn2g-nCBzCZ/,17902790093652450,"[{'timestamp': '2023-01-25T20:50:54+0000', 'te...",NaN,NaN,NaN
1,2023-01-24T19:20:12+0000,Full recipe in the video ⬆️\n\nThe one thing t...,50,2,VIDEO,REELS,NaN,https://www.instagram.com/reel/CnzzEKNhdLh/,17976757489963975,"[{'timestamp': '2023-01-24T20:04:16+0000', 'te...",https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,NaN
2,2023-01-23T18:52:42+0000,I used to talk a lot about the impact of my me...,17,0,VIDEO,REELS,https://video-sea1-1.cdninstagram.com/o1/v/t16...,https://www.instagram.com/reel/CnxK_ixhiuM/,17976666928950022,NaN,https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,NaN
3,2023-01-22T19:01:08+0000,𝐂𝐫𝐞𝐚𝐭𝐢𝐧𝐞\nCreatine is one of the most vastly r...,14,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/Cnunn04PpGs/,17918224373661214,NaN,NaN,NaN,NaN
4,2023-01-22T09:15:56+0000,Sunday evening thoughts 💭 \n\nAs you approach ...,72,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/Cntkpvuhj60/,17994459673604175,NaN,NaN,NaN,NaN


In [109]:
df_test_am.tail()

,timestamp,caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,thumbnail_url,comments.paging.cursors.after,comments.paging.next
20,2022-12-01T07:04:26+0000,Free December Deal 🔥🔥🔥\n\nYou won’t want to mi...,52,54,VIDEO,REELS,NaN,https://www.instagram.com/reel/Clnb7OHBMdZ/,17972337106905374,"[{'timestamp': '2022-12-02T23:42:22+0000', 'te...",https://scontent-sea1-1.cdninstagram.com/v/t51...,UVZAGRFMxRjNXRFJKTFhBdFdHdFZAlSGM1T1RCTlQyTkxR...,https://graph.facebook.com/v15.0/1797233710690...
21,2022-11-29T18:52:26+0000,Nutrition and Training Workshop 💪\n\nMy ethos ...,18,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/Cljjs0dhnUX/,17846185016881186,NaN,NaN,NaN,NaN
22,2022-11-28T18:57:08+0000,10 minutes to prepare and cook ✅\n\nServing se...,54,0,VIDEO,REELS,NaN,https://www.instagram.com/reel/Clg_JRyBbRU/,17878226135730844,NaN,https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,NaN
23,2022-11-27T19:20:45+0000,Mission accomplished ✅🎉\n\nTo say I’m proud wo...,165,53,VIDEO,REELS,NaN,https://www.instagram.com/reel/CledPjThEWL/,17965603229022919,"[{'timestamp': '2022-11-28T07:49:29+0000', 'te...",https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,NaN
24,2022-11-26T07:22:12+0000,One of the hardest parts of having online clie...,66,2,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/ClamUz7hU2O/,18191716690227443,"[{'timestamp': '2022-11-26T07:35:31+0000', 'te...",NaN,NaN,NaN


# Process post data

In [122]:
import re
def process_df_timestamp(df):
    """
    Convert dates in the json-derived dataframe into different formats.
    """
    df.reset_index(drop=True, inplace=True)
    regex_date = r'.+T'
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    df['time'] = df['timestamp'].dt.time
    df['hour'] = df['timestamp'].dt.hour
    return df
process_df_timestamp(df_test_am.tail(1).copy())

,timestamp,caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,thumbnail_url,comments.paging.cursors.after,comments.paging.next,date,year,month,day_of_week,time,hour
0,2022-11-26 07:22:12+00:00,One of the hardest parts of having online clie...,66,2,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/ClamUz7hU2O/,18191716690227443,"[{'timestamp': '2022-11-26T07:35:31+0000', 'te...",NaN,NaN,NaN,2022-11-26,2022,11,5,07:22:12,7


In [127]:
import re
import numpy as np
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
def preprocess_post_text(docs):
    """
    Prepare data from text documents for NLP:
    - Remove all the special characters
    - Remove all single characters
    - Substitute multiple spaces with single space
    - Convert to lowercase

    Parameters:
    docs (n x 1 array or string): Documents.

    Returns: Array of processed docs.
    """
    clean_docs = []
    wnl = WordNetLemmatizer()
    for doc in docs:
        try:
            # Split text into single words (also gets rid of extra white spaces)
            words = word_tokenize(doc)

            # Convert to lower case
            words = [word.lower() for word in words]

            # Lemmatize words (must be done after conversion to lower case)
            words = [wnl.lemmatize(word) for word in words]

            # Remove all single characters
            words = [word for word in words if len(word) > 2]
            
            # join words back together as a string
            words = ''.join([word+' ' for word in words])

            # Remove any URLs 
            words = re.sub(r'\w*\.+\w*', '', words) # Remove periods in middle of word
            words = re.sub(r'\w*/+\w*', '', words) # remove forward slash in middle of word
            words = re.sub(r'\w*/+\w*', '', words)

            # remove special characters
            words = ''.join([char for char in words if char not in string.punctuation])
            clean_docs.append(words)
        except: # In case value is nan
            clean_docs.append(doc)
        
    return np.array(clean_docs)

from sklearn.feature_extraction.text import CountVectorizer
from custom_nlp import *
def ig_post_preprocessing(df):
    df = process_df_timestamp(df)
    df['caption'] = preprocess_post_text(df['caption'])
    vect = CountVectorizer()
    vect.fit(df['caption'])
    vector = vect.transform(df['caption'])
    print('shape: ', vector.shape)
    vector_df = pd.DataFrame(vector.toarray(), columns=vect.get_feature_names_out())

    return df, vector_df

df, count_vector = ig_post_preprocessing(df_test_am.tail().copy())

shape:  (5, 266)


In [128]:
df

,timestamp,caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,thumbnail_url,comments.paging.cursors.after,comments.paging.next,date,year,month,day_of_week,time,hour
0,2022-12-01 07:04:26+00:00,free december deal 🔥🔥🔥 you won want miss want ...,52,54,VIDEO,REELS,NaN,https://www.instagram.com/reel/Clnb7OHBMdZ/,17972337106905374,"[{'timestamp': '2022-12-02T23:42:22+0000', 'te...",https://scontent-sea1-1.cdninstagram.com/v/t51...,UVZAGRFMxRjNXRFJKTFhBdFdHdFZAlSGM1T1RCTlQyTkxR...,https://graph.facebook.com/v15.0/1797233710690...,2022-12-01,2022,12,3,07:04:26,7
1,2022-11-29 18:52:26+00:00,nutrition and training workshop ethos help wom...,18,0,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/Cljjs0dhnUX/,17846185016881186,NaN,NaN,NaN,NaN,2022-11-29,2022,11,1,18:52:26,18
2,2022-11-28 18:57:08+00:00,minute prepare and cook serving serf 34 portio...,54,0,VIDEO,REELS,NaN,https://www.instagram.com/reel/Clg_JRyBbRU/,17878226135730844,NaN,https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,NaN,2022-11-28,2022,11,0,18:57:08,18
3,2022-11-27 19:20:45+00:00,mission accomplished say proud would understat...,165,53,VIDEO,REELS,NaN,https://www.instagram.com/reel/CledPjThEWL/,17965603229022919,"[{'timestamp': '2022-11-28T07:49:29+0000', 'te...",https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,NaN,2022-11-27,2022,11,6,19:20:45,19
4,2022-11-26 07:22:12+00:00,one the hardest part having online client not ...,66,2,IMAGE,FEED,https://scontent-sea1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/ClamUz7hU2O/,18191716690227443,"[{'timestamp': '2022-11-26T07:35:31+0000', 'te...",NaN,NaN,NaN,2022-11-26,2022,11,5,07:22:12,7


In [129]:
count_vector

,2022,2023,2024,34,3rd,7th,99,able,about,accomplished,...,within,without,woman,won,workhouse,workshop,would,year,you,your
0,1,2,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,7,2
1,0,0,1,0,0,1,1,0,0,0,...,0,1,1,0,1,1,0,1,4,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,0,1,0,1,...,1,1,0,0,0,0,2,0,2,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


SH 2023-01-25 22:54
Next steps:
* Create regular expression that replaces all tokens starting with a digit into '[number]'
* Allow optional arguments to be entered into the CountVectorizer instantiation:
    * stop_words

# *End of Page*